In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Concatenate, UpSampling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model

def create_unet_model(input_size=(128, 128, 3), num_classes=1, segmentation_type='binary'):
    # Base preentrenada de MobileNetV2
    base_model = MobileNetV2(input_shape=input_size, include_top=False)
    
    # Capas de interés del modelo base para la conexión
    layer_names = [
        'block_1_expand_relu',   # 64x64
        'block_3_expand_relu',   # 32x32
        'block_6_expand_relu',   # 16x16
        'block_13_expand_relu',  # 8x8
        'block_16_project',      # 4x4
    ]
    layers = [base_model.get_layer(name).output for name in layer_names]

    # Crear el modelo de extracción de características
    down_stack = Model(inputs=base_model.input, outputs=layers)
    down_stack.trainable = False

    # Definición de la parte ascendente del U-Net
    up_stack = [
        UpSampling2D((2, 2)),  # 4x4 -> 8x8
        UpSampling2D((2, 2)),  # 8x8 -> 16x16
        UpSampling2D((2, 2)),  # 16x16 -> 32x32
        UpSampling2D((2, 2)),  # 32x32 -> 64x64
    ]

    inputs = tf.keras.layers.Input(shape=input_size)
    x = inputs

    # Descenso en el U-Net
    skips = down_stack(x)
    x = skips[-1]
    skips = reversed(skips[:-1])

    # Ascenso en el U-Net y concatenación con las capas de salto
    for up, skip in zip(up_stack, skips):
        x = up(x)
        concat = Concatenate()
        x = concat([x, skip])

    # Capa final de convolución dependiendo del tipo de segmentación
    if segmentation_type == 'binary':
        x = Conv2D(1, 3, padding='same', activation='sigmoid')(x)
        loss = 'binary_crossentropy'
    else:
        x = Conv2D(num_classes, 3, padding='same', activation='softmax')(x)
        loss = 'sparse_categorical_crossentropy'

    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    return model

# Configuración del modelo
model = create_unet_model(segmentation_type='nobynary')

In [15]:
import tensorflow as tf
import numpy as np
import os
import cv2

IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

TRAIN_PATH = '/home/luishuingo/semantics_segmentation/Floor-Segmentationv1/train'
TEST_PATH = '/home/luishuingo/semantics_segmentation/Floor-Segmentationv1/test'
VALID_PATH = '/home/luishuingo/semantics_segmentation/Floor-Segmentationv1/valid/'

X_train = np.zeros((len(os.listdir(TRAIN_PATH)), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(os.listdir(TRAIN_PATH)), IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)

print('Resizing training images and masks')

for n, id_ in enumerate(os.listdir(TRAIN_PATH)):
    
    # Load and resize the training image
    img = cv2.imread(os.path.join(TRAIN_PATH, id_))  # Read the image using OpenCV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB if necessary
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_CUBIC)  # Resize the image
    X_train[n] = img

    # Load and resize the corresponding mask
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)
    mask_ = cv2.imread(os.path.join(TRAIN_PATH, id_.replace('.jpg', '_mask.png')), cv2.IMREAD_GRAYSCALE)  # Read the mask as grayscale
    mask_ = cv2.resize(mask_, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_CUBIC)  # Resize the mask
    mask_ = np.expand_dims(mask_, axis=-1)
    mask = np.maximum(mask, mask_)
    Y_train[n] = mask

# Now we do the same for the test set
X_test = np.zeros((len(os.listdir(TEST_PATH)), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_test = np.zeros((len(os.listdir(TEST_PATH)), IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)

print('Resizing test images and masks')

for n, id_ in enumerate(os.listdir(TEST_PATH)):

    # Load and resize the training image
    img = cv2.imread(os.path.join(TEST_PATH, id_))  # Read the image using OpenCV
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB if necessary
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_CUBIC)  # Resize the image
    X_test[n] = img

    # Load and resize the corresponding mask
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)
    mask_ = cv2.imread(os.path.join(TEST_PATH, id_.replace('.jpg', '_mask.png')), cv2.IMREAD_GRAYSCALE)  # Read the mask as grayscale
    mask_ = cv2.resize(mask_, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_CUBIC)  # Resize the mask
    mask_ = np.expand_dims(mask_, axis=-1)
    mask = np.maximum(mask, mask_)
    Y_test[n] = mask

# Now we do the same for the validation set

X_valid = np.zeros((len(os.listdir(VALID_PATH)), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_valid = np.zeros((len(os.listdir(VALID_PATH)), IMG_HEIGHT, IMG_WIDTH, 1), dtype=bool)

print('Resizing validation images and masks')

Resizing training images and masks
Resizing test images and masks
Resizing validation images and masks


In [16]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_8 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 model_6 (Functional)        [(None, 64, 64, 96),         1841984   ['input_8[0][0]']             
                              (None, 32, 32, 144),                                                
                              (None, 16, 16, 192),                                                
                              (None, 8, 8, 576),                                                  
                              (None, 4, 4, 320)]                                                  
                                                                                            

In [17]:
# Model checkpoint
checkpointer = tf.keras.callbacks.ModelCheckpoint('model_for_nuclei.h5', verbose=1, save_best_only=True)

callbacks = [
              tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
              tf.keras.callbacks.TensorBoard(log_dir='logs')]


results = model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), batch_size=16, epochs=100, callbacks=callbacks,)


Epoch 1/100


ValueError: in user code:

    File "/home/luishuingo/anaconda3/envs/tf_seg/lib/python3.9/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/home/luishuingo/anaconda3/envs/tf_seg/lib/python3.9/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/luishuingo/anaconda3/envs/tf_seg/lib/python3.9/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/home/luishuingo/anaconda3/envs/tf_seg/lib/python3.9/site-packages/keras/src/engine/training.py", line 1131, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/home/luishuingo/anaconda3/envs/tf_seg/lib/python3.9/site-packages/keras/src/engine/training.py", line 1225, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_wei